In [1]:
from scapy.all import rdpcap, TCP
from datetime import datetime

In [4]:
def analyze_pcap(file_path):
    packets = rdpcap(file_path)
    
    total_bytes = 0
    data_bytes = 0
    total_packets = 0
    lost_packets = 0
    max_packet_size = 0
    
    ack_numbers = set()
    seq_numbers = set()
    
    start_time = None
    end_time = None
    
    for pkt in packets:
        if TCP in pkt and pkt.time:
            total_packets += 1
            packet_size = len(pkt)
            total_bytes += packet_size
            
            if pkt[TCP].flags & 0x02:  # SYN flag
                start_time = pkt.time
            
            if pkt[TCP].flags & 0x01 and pkt[TCP].flags & 0x10:  # FIN-ACK
                end_time = pkt.time
            
            if pkt[TCP].seq not in seq_numbers:
                seq_numbers.add(pkt[TCP].seq)
                data_bytes += len(pkt[TCP].payload)
            
            if pkt[TCP].ack not in ack_numbers:
                ack_numbers.add(pkt[TCP].ack)
                
            max_packet_size = max(max_packet_size, packet_size)
    
    duration = end_time - start_time if start_time and end_time else 0
    
    # Throughput (total bytes transferred / duration)
    throughput = total_bytes / duration if duration > 0 else 0
    
    # Goodput (total useful data bytes / duration)
    goodput = data_bytes / duration if duration > 0 else 0
    
    # Packet loss rate
    expected_packets = len(seq_numbers)
    lost_packets = expected_packets - len(ack_numbers)
    packet_loss_rate = (lost_packets / expected_packets) * 100 if expected_packets > 0 else 0
    
    return {
        "Duration (seconds)": duration,
        "Throughput (bytes/sec)": throughput,
        "Goodput (bytes/sec)": goodput,
        "Packet Loss Rate (%)": packet_loss_rate,
        "Maximum Packet Size (bytes)": max_packet_size
    }

`config0 - 0, 0` => Nagle Dis, Delayed Ack<br>
`config1 - 0, 1` => Nagle Dis, No Delayed Ack<br>
`config2 - 1, 0` => Nagle En, Delayed Ack<br>
`config3 - 1, 1` => Nagle En, No Delayed Ack

In [5]:
file_path = './config0.pcap'
results = analyze_pcap(file_path)
for metric, value in results.items():
    print(f"{metric}: {value}")

Duration (seconds): 0.062339
Throughput (bytes/sec): 235358.2829368453135276472192
Goodput (bytes/sec): 65705.25674136575819310543961
Packet Loss Rate (%): 47.22222222222222
Maximum Packet Size (bytes): 106


In [6]:
file_path = './config1.pcap'
results = analyze_pcap(file_path)
for metric, value in results.items():
    print(f"{metric}: {value}")

Duration (seconds): 0.060386
Throughput (bytes/sec): 299804.5904679892690358692412
Goodput (bytes/sec): 67167.88659623091445036929090
Packet Loss Rate (%): 0.0
Maximum Packet Size (bytes): 106


In [7]:
file_path = './config2.pcap'
results = analyze_pcap(file_path)
for metric, value in results.items():
    print(f"{metric}: {value}")

Duration (seconds): 0.059575
Throughput (bytes/sec): 85639.94964330675618967687789
Goodput (bytes/sec): 68753.67184221569450272765422
Packet Loss Rate (%): 27.27272727272727
Maximum Packet Size (bytes): 1514


In [8]:
file_path = './config3.pcap'
results = analyze_pcap(file_path)
for metric, value in results.items():
    print(f"{metric}: {value}")

Duration (seconds): 0.060061
Throughput (bytes/sec): 297031.3514593496611778025674
Goodput (bytes/sec): 67531.34313447994538885466442
Packet Loss Rate (%): 0.0
Maximum Packet Size (bytes): 146
